<a href="https://colab.research.google.com/github/deconasser/ComputerVisionCourse/blob/main/AIO/feedforward_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as tranforms
from torch.utils.data import DataLoader

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

**Hyper params**

In [5]:
input = 28*28
hidden_layers = 500
num_classes = 10
num_epochs = 10
batch_size = 100
learning_rate = 0.001

**MNIST DATASET**

In [6]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

--2024-08-01 07:25:38--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2024-08-01 07:25:39--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [              <=>   ]  33.20M  10.8MB/s    in 3.1s    

2024-08-01 07:25:42 (10.8 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [72]:
transform = tranforms.ToTensor()
train_dataset = torchvision.datasets.MNIST(root="./", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./", train=False, transform=transform)

In [8]:
#Data Loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset= test_dataset, batch_size=batch_size, shuffle=False)

**FCN**

In [73]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)

    return out

**Create Model**

In [74]:
model = NeuralNetwork(input, hidden_layers, num_classes).to(device)

**Optimizer**

In [76]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

**TRAIN MODEL**

In [77]:
total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Move tensor to the configured device
    images = images.reshape(-1, input).to(device)
    labels = labels.to(device)

    # forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # backward and optimizer
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
          print ('Epoch [{}/{}], Step [{}/{}], Loss: {}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [100/600], Loss: 2.2851898670196533
Epoch [1/10], Step [200/600], Loss: 2.2744503021240234
Epoch [1/10], Step [300/600], Loss: 2.2416067123413086
Epoch [1/10], Step [400/600], Loss: 2.2178542613983154
Epoch [1/10], Step [500/600], Loss: 2.2069058418273926
Epoch [1/10], Step [600/600], Loss: 2.1869559288024902
Epoch [2/10], Step [100/600], Loss: 2.1350128650665283
Epoch [2/10], Step [200/600], Loss: 2.12990665435791
Epoch [2/10], Step [300/600], Loss: 2.1011931896209717
Epoch [2/10], Step [400/600], Loss: 2.0986249446868896
Epoch [2/10], Step [500/600], Loss: 2.063831090927124
Epoch [2/10], Step [600/600], Loss: 2.0116872787475586
Epoch [3/10], Step [100/600], Loss: 1.942845344543457
Epoch [3/10], Step [200/600], Loss: 1.9968886375427246
Epoch [3/10], Step [300/600], Loss: 1.9339183568954468
Epoch [3/10], Step [400/600], Loss: 1.8699036836624146
Epoch [3/10], Step [500/600], Loss: 1.9058417081832886
Epoch [3/10], Step [600/600], Loss: 1.8402024507522583
Epoch [4/10], 

**Test model**

In [78]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images = images.reshape(-1, input).to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print('Accuracy: {}'.format(100 * correct / total))

Accuracy: 84.1


In [79]:
torch.save(model.state_dict(), "NN_model.ckpt")